In [ ]:
from datetime import datetime
import requests
import configparser
import hmac
import hashlib
import base64
import json

config = configparser.ConfigParser()
config.read('config.ini')
access_key = config.get("okx", "api_key")
secret_key = config.get("okx", "secret_key")
access_passphrase = config.get("okx", "password")
access_timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+"Z"
method = "POST"
request_path = "/api/v5/mktplace/nft/ordinals/trade-history"
body = json.dumps({"slug":"8888", "limit":10})
prehash = access_timestamp + method + request_path + body
signature = hmac.new(secret_key.encode(), prehash.encode(), hashlib.sha256).digest()
access_sign = base64.b64encode(signature).decode()
headers = {
            'OK-ACCESS-KEY': access_key,
            'OK-ACCESS-TIMESTAMP': access_timestamp,
            'OK-ACCESS-PASSPHRASE': access_passphrase,
            'OK-ACCESS-SIGN': access_sign,
            'Content-Type': 'application/json'
        }
response = requests.post("https://www.okx.com/api/v5/mktplace/nft/ordinals/trade-history", headers=headers, data=body)

In [ ]:
import pandas as pd

method = "POST"
request_path = "/api/v5/mktplace/nft/ordinals/trade-history"
url = "https://www.okx.com" + request_path
body = json.dumps({"slug":"rats", "limit":100})
OK_ACCESS_KEY = config.get("okx", "api_key")
OK_ACCESS_TIMESTAMP = datetime.utcnow().\
    strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+"Z"
OK_ACCESS_PASSPHRASE = config.get("okx", "password")
prehash = OK_ACCESS_TIMESTAMP + method + request_path + body
signature = hmac.new(
            secret_key.encode(), 
            prehash.encode(), 
            hashlib.sha256
        ).digest()
        
OK_ACCESS_SIGN = base64.b64encode(signature).decode()

headers = {
    'OK-ACCESS-KEY': OK_ACCESS_KEY,
    'OK-ACCESS-TIMESTAMP': OK_ACCESS_TIMESTAMP,
    'OK-ACCESS-PASSPHRASE': OK_ACCESS_PASSPHRASE,
    'OK-ACCESS-SIGN': OK_ACCESS_SIGN,
    'Content-Type': 'application/json'
}
response = requests.post(url, headers=headers, data=body)
try:
    cursor = response.json()["data"]["cursor"]
    data = response.json()["data"]["data"]
except:
    raise RuntimeError(response.text)
trade_history = pd.DataFrame(data)
while cursor != "":
    body = json.loads(body)
    body["cursor"] = cursor
    body = json.dumps(body)
    OK_ACCESS_KEY = config.get("okx", "api_key")
    OK_ACCESS_TIMESTAMP = datetime.utcnow().\
        strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]+"Z"
    OK_ACCESS_PASSPHRASE = config.get("okx", "password")
    prehash = OK_ACCESS_TIMESTAMP + method + request_path + body
    signature = hmac.new(
                secret_key.encode(), 
                prehash.encode(), 
                hashlib.sha256
            ).digest()
            
    OK_ACCESS_SIGN = base64.b64encode(signature).decode()

    headers = {
        'OK-ACCESS-KEY': OK_ACCESS_KEY,
        'OK-ACCESS-TIMESTAMP': OK_ACCESS_TIMESTAMP,
        'OK-ACCESS-PASSPHRASE': OK_ACCESS_PASSPHRASE,
        'OK-ACCESS-SIGN': OK_ACCESS_SIGN,
        'Content-Type': 'application/json'
    }
    
    response = requests.post(url, headers=headers, data=body)
    try:
        cursor = response.json()["data"]["cursor"]
        data = response.json()["data"]["data"]
    except:
        raise RuntimeError(response.text)
    trade_history = pd.concat([trade_history, pd.DataFrame(data)])
    print(trade_history.iloc[-1,])

In [1]:
from classes import Token
from configparser import ConfigParser

config = ConfigParser()
config.read("config.ini")
okx_api_key = config.get("okx", "api_key")
okx_secret_key = config.get("okx", "secret_key")
okx_password = config.get("okx", "password")
ordi = Token("ordi", True)

In [3]:
ordi.get_floor_listing(
    okx_api_key,
    okx_secret_key,
    okx_password
)

{'amount': 500.0,
 'unit_price': 0.001765,
 'total_price': 0.8825,
 'listing_time': datetime.datetime(2023, 12, 6, 10, 34, 51)}

In [4]:
ordi.get_trade_history(
    okx_api_key,
    okx_secret_key,
    okx_password
)

,amount,unit_price,total_price,time
0,10.0,0.001680,0.016800,2023-12-25 15:43:10
1,10.0,0.001850,0.018500,2023-12-25 15:43:10
2,10.0,0.001680,0.016800,2023-12-25 15:43:10
3,2.0,0.001265,0.002530,2023-12-25 15:43:10
4,20.0,0.001700,0.034000,2023-12-25 15:43:10
...,...,...,...,...
795,50.0,0.001500,0.075000,2023-12-14 08:55:34
796,50.0,0.001509,0.075444,2023-12-14 08:55:34
797,25.0,0.001460,0.036500,2023-12-14 08:55:34
798,75.0,0.001490,0.111750,2023-12-14 08:55:34


In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

plt.plot(ordi.trade_history["time"], ordi.trade_history["unit_price"])
plt.yticks(np.linspace(ordi.trade_history["unit_price"].min(), ordi.trade_history["unit_price"].max(), 10))